In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pyspark-linear-regression/Linear_regression_dataset.csv


In [2]:
!pip install findspark
!pip install pyspark

     |████████████████████████████████| 281.4 MB 31 kB/s              
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 198 kB 63.3 MB/s            
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=bf5c2a22d4fd72dc6078b49b59054b16ca7054beb8a3609647b1242bac2f0b72
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.4
    Uninstalling py4j-0.10.9.4:
      Successfully uninstalled py4j-0.10.9.4


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('liner-regress').getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/08 11:03:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.sql.functions import corr
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [5]:
#Import Libs
df = spark.read.csv('../input/pyspark-linear-regression', header=True, inferSchema=True)
df.head()

Row(var_1=734, var_2=688, var_3=81, var_4=0.328, var_5=0.259, output=0.418)

In [6]:
#check the shape of the data
df.count(), len(df.columns)

(1232, 6)

In [7]:
df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)



In [8]:
df.describe().show()

+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|            var_1|            var_2|             var_3|               var_4|               var_5|             output|
+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|             1232|             1232|              1232|                1232|                1232|               1232|
|   mean|715.0819805194806|715.0819805194806| 80.90422077922078|  0.3263311688311693| 0.25927272727272715|0.39734172077922014|
| stddev| 91.5342940441652|93.07993263118064|11.458139049993724|0.015012772334166148|0.012907228928000298|0.03326689862173776|
|    min|              463|              472|                40|               0.277|               0.214|              0.301|
|    max|             1009|             1103|               116|               0.373|               0.294|     

In [9]:
df.select(corr('var_1', 'output')).show()

+-------------------+
|corr(var_1, output)|
+-------------------+
| 0.9187399607627283|
+-------------------+



In [10]:
df.select(corr('var_2', 'output')).show()

+-------------------+
|corr(var_2, output)|
+-------------------+
|0.43652698913681093|
+-------------------+



In [11]:
df.columns

['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'output']

In [12]:
vec_assembler = VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5']
                                ,outputCol='features')

In [13]:
featuredf = vec_assembler.transform(df)
featuredf.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)
 |-- features: vector (nullable = true)



In [14]:
featuredf.show(5,truncate=False)

+-----+-----+-----+-----+-----+------+------------------------------+
|var_1|var_2|var_3|var_4|var_5|output|features                      |
+-----+-----+-----+-----+-----+------+------------------------------+
|734  |688  |81   |0.328|0.259|0.418 |[734.0,688.0,81.0,0.328,0.259]|
|700  |600  |94   |0.32 |0.247|0.389 |[700.0,600.0,94.0,0.32,0.247] |
|712  |705  |93   |0.311|0.247|0.417 |[712.0,705.0,93.0,0.311,0.247]|
|734  |806  |69   |0.315|0.26 |0.415 |[734.0,806.0,69.0,0.315,0.26] |
|613  |759  |61   |0.302|0.24 |0.378 |[613.0,759.0,61.0,0.302,0.24] |
+-----+-----+-----+-----+-----+------+------------------------------+
only showing top 5 rows



# Modelling

In [15]:
model_df = featuredf.select('features', 'output')
model_df.show(5, False)

+------------------------------+------+
|features                      |output|
+------------------------------+------+
|[734.0,688.0,81.0,0.328,0.259]|0.418 |
|[700.0,600.0,94.0,0.32,0.247] |0.389 |
|[712.0,705.0,93.0,0.311,0.247]|0.417 |
|[734.0,806.0,69.0,0.315,0.26] |0.415 |
|[613.0,759.0,61.0,0.302,0.24] |0.378 |
+------------------------------+------+
only showing top 5 rows



In [16]:
#shape of the modelling df
print((model_df.count()), (len(model_df.columns)))

1232 2


In [17]:
#train test split
traindf, testdf = model_df.randomSplit([0.7, 0.3])
print((traindf.count()), (len(traindf.columns)))

866 2


In [18]:
print((testdf.count()), (len(testdf.columns)))

366 2


In [19]:
lin_reg = LinearRegression(labelCol='output')
lr_model = lin_reg.fit(traindf)

22/04/08 11:04:13 WARN Instrumentation: [004c2acf] regParam is zero, which might cause numerical instability and overfitting.


In [20]:
print(lr_model.coefficients)

[0.00035099389979091053,5.048309429481829e-05,0.00012828411219195914,-0.64181295921011,0.45518586498134717]


In [21]:
print(lr_model.intercept)

0.19184779676850036


In [22]:
training_predictions = lr_model.evaluate(traindf)

In [23]:
training_predictions.r2

0.8715539052785934

In [24]:
test_predictions = lr_model.evaluate(testdf)
test_predictions.r2

0.862352852637478

In [25]:
print(test_predictions.meanSquaredError)

0.00016103148295356978
